In [1]:
import os
import sys
import pandas as pd
import pickle
import json
import socket as s
import threading
import traceback
from socketserver import ThreadingTCPServer
import logging

In [2]:
IP_address = s.gethostbyname(s.gethostname())
#IP = ['10.200.147.198','10.200.0.169','10.200.132.105']

# Enter the IPs of the servers in order: [Server1   Server 2   Server 3]
#IP = ['10.200.133.129','10.200.133.129','10.200.133.129']
IP = ['192.168.0.9', '192.168.0.9', '192.168.0.9']
port1 = 8088
# This are the ports of the respective srvers port: [Server1   Server 2   Server 3] at whiich master node will connect 
port = [3333, 3334, 3335]
buffer = 1024
max_connections = 1000
client_buffer = 1024
# To send encoding message utf-8 encoding format is used.
encoding_format = 'utf-8'
print(IP_address)

127.0.0.1


In [3]:
#creating configuration file for logging the user activity
root_logger = logging.getLogger(__name__)
root_logger.setLevel(logging.INFO)
handler = logging.FileHandler('master_logs.log', 'w', 'utf-8')
handler.setLevel(logging.INFO)
root_logger.addHandler(handler)
formatter = logging.Formatter('%(asctime)s  %(name)s  %(levelname)s: %(message)s')
handler.setFormatter(formatter)

In [4]:
validation = pd.DataFrame(pd.read_excel("validation.xlsx"))

In [5]:
###########################################################
##       Validate the user/client
##########################################################

def validateLogin(username, password):
    available_files = []
    for i in range(len(validation["username"])):
        if validation.loc[i][0] == username:
            if validation.loc[i][1] == password:
                output = "success"
                username = username
                root_logger.info("%s logged in ", username)
#                         readFiles()
            else:
                output = "not success"
                print("wrong password")
                root_logger.error("Incorrect Password")
            break
        else:
            output = "not user"
            root_logger.error("Invalid User")
    return output


In [6]:
####################################################
# Get the permission of the client fron Excel Sheet
####################################################

def get_permission(username, file_name):
    
    #  Give the actual path of the permission's excel sheet present in your devise in variable given below 
    
    file_path = r'permissions.xlsx'
    
    permissions_dataframe = pd.read_excel(file_path, index_col=0)
    
    permission = permissions_dataframe.loc[username][file_name]

    return permission


In [7]:
##################################################
##       Change the client permision
##################################################


def change_permission(file_name, user_name_to_change_permission, updated_permission):
    
    file_path = r'permissions.xlsx'
    permissions_dataframe = pd.read_excel(file_path, index_col=0)
    
    permissions_dataframe.loc[user_name_to_change_permission][file_name] = updated_permission
    
    lock.acquire()
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="w", if_sheet_exists='replace') as writer:
        permissions_dataframe.to_excel(writer, 'Sheet1', index=True)

    lock.release()
    
    return "Permission updated"
    

'''def change_permission():
    username= str(input())
    file_name= str(input())

    lock = threading.Lock()

    #permission = get_permission(username, file_name)
    
    #  Give the actual path of the permission's excel sheet present in your devise in variable given below
    
    file_path = r'permissions.xlsx'
    permissions_dataframe = pd.read_excel(file_path, index_col=0)

    permission = permissions_dataframe.loc[username][file_name]

    if permission == "owner":
        print("Enter the user whose permission needs to be changed")
        user_name_to_change_permission = str(input())
        print("Enter the updated permission")
        updated_permission = str(input())

        permissions_dataframe.loc[user_name_to_change_permission][file_name] = updated_permission

        print(permissions_dataframe)

        lock.acquire()
        with pd.ExcelWriter(file_path, engine="openpyxl", mode="w", if_sheet_exists='replace') as writer:
            permissions_dataframe.to_excel(writer, 'Sheet1', index=True)

        lock.release()
    else:
        print("You are not authorized to modify the permission")'''

'def change_permission():\n    username= str(input())\n    file_name= str(input())\n\n    lock = threading.Lock()\n\n    #permission = get_permission(username, file_name)\n    \n    #  Give the actual path of the permission\'s excel sheet present in your devise in variable given below\n    \n    file_path = r\'permissions.xlsx\'\n    permissions_dataframe = pd.read_excel(file_path, index_col=0)\n\n    permission = permissions_dataframe.loc[username][file_name]\n\n    if permission == "owner":\n        print("Enter the user whose permission needs to be changed")\n        user_name_to_change_permission = str(input())\n        print("Enter the updated permission")\n        updated_permission = str(input())\n\n        permissions_dataframe.loc[user_name_to_change_permission][file_name] = updated_permission\n\n        print(permissions_dataframe)\n\n        lock.acquire()\n        with pd.ExcelWriter(file_path, engine="openpyxl", mode="w", if_sheet_exists=\'replace\') as writer:\n          

In [8]:
##################################################
##   set the initial permision of the client
##################################################

def set_permission_for_new_file(file_name, username):

    lock = threading.Lock()
    
    #  Give the actual path of the permission's excel sheet present in your devise in variable given below
    
    file_path = r'permissions.xlsx'
    permissions_dataframe = pd.read_excel(file_path, index_col=0)
    user_permission = []
    print(username)
    for user in permissions_dataframe.index:
        if user == username:
            prm = 'owner'
        else:
            prm = 'not'
        user_permission.append(prm)

    permissions_dataframe[file_name] = user_permission
    root_logger.info(f"file permission for {file_name} updated for user {username} ")
    print(permissions_dataframe)

    lock.acquire()
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="w", if_sheet_exists='replace') as writer:
        permissions_dataframe.to_excel(writer, 'Sheet1', index=True)

    lock.release()
    return "file created"

In [9]:
def perform_operation(conn, addr):
    
#  Getting the data from the client 

    while True:
        data1 = conn.recv(buffer)
        content = pickle.loads(data1)
        operation = content[0]

        
        def connect_all_server():
########################################################################################
# This section is for looping the connection to all server to make update in all server.
########################################################################################
            for i in range(3): 
                with s.socket(s.AF_INET, s.SOCK_STREAM) as client:
                    client.connect((IP[i], port[i]))
                    client.send(data1)
                    data = client.recv(client_buffer)
            return data

#####################################################################
# Authentication of the client/user
#####################################################################

        if operation == "validation":
            valid = validateLogin(content[1], content[2])
            print(valid)
            conn.send(valid.encode())
            
            
#####################################################################
# Authentication of the client/user
##################################################################### 

            
        else:
            if operation == "createfile":
                
                data = set_permission_for_new_file(content[3], content[1])
                data = connect_all_server()
                conn.send(data)
                
            elif operation == "writefile":
#             or "":
                permission = get_permission(content[1], content[3])
                print("User : " + content[1] + " || "+ " File : " + content[3] + " || "+ " Authorship : "  + permission)
                
            
                if permission == "owner" or permission == "write":
                    data = connect_all_server()
                    root_logger.info(f"Write operation performed on {content[3]} with user {content[1]}")
                    conn.send(data)
                else:
                    data = "You have no permision to write"
                    root_logger.error("Insufficient permission to write")
                    conn.send(data.encode())
                    
            elif operation == "deletefile": 
#             or operation == "renamefile":
                
                permission = get_permission(content[1], content[3])
                root_logger.info(f"Delete operation performed on {content[3]} with user {content[1]}" )
                print("User : " + content[1] + " || "+ " File : " + content[3] + " || "+ " Authorship : "  + permission)
            
                if permission == "owner":
                    data = connect_all_server()
                    conn.send(data)
                else:
                    data = "You are not the owner of this file"
                    root_logger.error("Insufficient permission to delete")
                    conn.send(data.encode())
            
            elif operation == "changePermission":
                permission = get_permission(content[1], content[2])
                
                if permission == "owner":
                    root_logger.info(f"Permission changed for {content[2]} on file {content[3]}")
                    data = change_permission(content[2], content[3], content[4])
                    conn.send(data)
                else:
                    data = "You are not authorized to change permission"
                    root_logger.error("You are not authorized to change permission")
                    conn.send(data.encode())
                
            
            else:
                data = connect_all_server()    
                conn.send(data)    
            
    print(f"[Disconnect] Disconnected {addr} client")
    conn.close()

In [10]:
def connect_server():
    #creating a new socket
    #socket functions takes as argument the socket family and socket type
    with s.socket(s.AF_INET, s.SOCK_STREAM) as server:
        server.setsockopt(s.SOL_SOCKET, s.SO_REUSEADDR, 1)
        try:
            server.bind((IP_address, port1))
        except:
            print("Error in binding the socket")
            root_logger.error("Error in binding the socket")
            sys.exit()
        thread_list = []
        server.listen(max_connections) #queues upto 10 requests
        print(f"Server listening upto {max_connections - (threading.activeCount()-1)} connections\n")
        while True:
            connection, client_addr = server.accept()
            try:
                conn_thread = threading.Thread(target=perform_operation, args=(connection, client_addr))
                conn_thread.start()
                #thread_list.append(conn_thread)
                print(f"Total Active connections {threading.activeCount()-1}")
            except:
                print("Error in starting the thread")
                root_logger.error("Error in starting the thread")
                traceback.print_exc()

In [ ]:
if __name__ == "__main__":
    connect_server()

Server listening upto 996 connections

Total Active connections 5
success
payodhi
            djp payodhifile dharanifile suryafile chandufile   temp  pay12  \
Username                                                                     
dhruv     owner        read       write      not        read  owner    not   
payodhi   write       owner        read     write        not    not    not   
dharani    read       write       owner      not       write    not    not   
surya       not        read       write     owner      write    not    not   
chandu     read         not         not     write      owner    not    not   
random      not        read        read      read        not    not  owner   

         pay123  abc12  
Username                
dhruv       not    not  
payodhi     not  owner  
dharani     not    not  
surya       not    not  
chandu      not    not  
random    owner    not  
payodhi
            djp payodhifile dharanifile suryafile chandufile   temp  pay12  \
Usernam

In [ ]:
### Encryption Algorithm

import os
from cryptography import fernet
from cryptography.fernet import Fernet

class Encrypter:

    def createKey(self):
        

        # creating a variable key_generated to check if key is generated
        key_generated=False

        while key_generated==False:

            # Creating A Try Except Block when key folder is not found.

            try:
                # If the function doesn't raise an exception it creates a file key.key and writes the encryption key in it.
                
                with open('key/key.key','wb') as f:
                    f.write(Fernet.generate_key())
                    f.close()

                print('Key Generated.')
                key_generated=True

     
            except:
                # The program raises an exception when it doesn't find the key
                # folder so it creates one instead of throwing an error.

                os.makedirs('./key')

    def encryptFile(self,file):
        '''This Function Encrypts the file which is given as the parameter'''

        # Creating filename variable so that the function extracting only the filename from the file location
        filename=file.split('\\')[-1]

        # Reading the file to get the data from the file 
        # so that it can be encrypted in another file
        with open(file,'rb') as f:
            data=f.read()
            f.close()


        # Using a Try an Except block to get the encryption key

        try:
            # Tries to to read the key 
            with open('key/key.key','rb') as f:
                key=f.read()
                f.close()

        except:
            # The function raises an exception if the key is not found

            print('Key Not Found Please Generate a Key')


        # Creating a file_written variable to check if the file is written
        file_written=False

        # While file is not written 
        while file_written==False:

            # Tries to make a directory if the directory exists then
            # creates the file in that directory  

            try:
                os.makedirs('./Encrypted Files')
                
            except FileExistsError:
                with open('Encrypted Files/Encrypted {}'.format(filename),'wb') as f:
                    f.write(Fernet(key).encrypt(data))
                    f.close()
                os.remove(file)
                print('File Encrypted')
                file_written=True

    def decryptFile(self,encryptedfile):
        '''This Function Decrypts the file'''

        
        filename=encryptedfile.split('\\')[-1]
        filename=filename.split(' ')[-1]

       
        with open(encryptedfile,'rb') as f:
            data=f.read()
            f.close()

        
        with open('key/key.key','rb') as f:
            key=f.read()
            f.close()
        
        
        file_written=False

        

        while file_written==False:
        
            # Tries to make the directory first if directory exist then 
            # it creates the original file in the decrypted files directory
            
            try:
            
                os.makedirs('./Decrypted Files')
            
            except FileExistsError:


            
                with open('Decrypted Files/{}'.format(filename),'wb') as f:
                    f.write(Fernet(key).decrypt(data))
                    f.close()
            
                print('File Decrypted.')
                file_written=True